In [1]:
using JuMP, Gurobi
using CSV, LinearAlgebra, DataFrames
using Plots

### Data

In [2]:
#load social and ecological data and orgnize 
FL = repeat([Float64], inner=15)
dtype = append!([String], FL);

regional_EF = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/new_EF_SC1.csv",header=1,delim=",", types=dtype) |> DataFrame    
capacity = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/capacity2.csv",header=1,delim=",", types=dtype) |> DataFrame    
distance = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/distance.csv",header=1,delim=",") |> DataFrame 
LCA_model = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/LCA_model2.csv",header=1,delim=",") |> DataFrame 
D_Dsoc = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/D_Dsoc1.csv",header=1,delim=",") |> DataFrame
GDP = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/social/GDP.csv",header=1,delim=",") |> DataFrame;
emi_sink = CSV.File("C:/Users/bourg/.julia/environments/batterySC/Li-battery-SC/data/SC_regional/emission_sink1.csv",header=1,delim=",") |> DataFrame;

In [3]:
cell_demand = 0.001*164.98*(1.3m*1e6)*2           # annual demand of Li battery for tesla (1.369M EV/yr, ~2 NMC111 pack/EV, 164.98 kg/pack (35kwh/pack), 80~100 kWh per EV)

global_sink = 1.099e10                        # global pub (ocean) CO2 sequestration (ton/yr)
global_sink_tot = 2.236e10                  # global total (ocean+land) CO2 sequestration (ton/yr)
global_emi = 3.53e10                          # global CO2 emission (ton/yr)
global_gdp = 96882e9                          # 2021 global GDP ($/yr)
es_ratio = global_sink/global_emi
es_ratio_tot = global_sink_tot/global_emi
emission_c = emi_sink[!, "emission"]          # national CO2 emission (ton/yr)
sink_c = emi_sink[!, "sink ton/yr"]           # national CO2 sink (ton/yr)

D = D_Dsoc[!, "D"]          # national CO2 emission (ton/yr)
Dsoc = D_Dsoc[!, "Dsoc ton/yr"]           # national CO2 sink (ton/yr)

EF_trans = 1.005/10000                        # ton CO2/km*ton (The average freight truck in the U.S. emits 161.8 grams of CO2 per ton-mile)
process = LCA_model[!,"process"]
countries = capacity[!,"country"]
ncty = size(countries,1)                          # No. of countries
nproc = size(process,1);                          # No. of processes 

mkt_loc = findfirst(isequal("United States"), countries)
mkt_proc = findfirst(isequal("battery"), process)

# seperate model
cathode = 1:4
cell = 5:10
noncell = [12,13]
battery = [11,14]
scaler = LCA_model[!,"scaler"]
price = LCA_model[!,"price (usd/ton product)"]
vGDP = GDP[!,"GDP usd"];

In [4]:
up_cath = scaler[1:4] * scaler[5] * scaler[11]
up_cell = scaler[5:10] * scaler[11]
cell_sef = scaler[11]
up_noncell = scaler[12:13] * scaler[14]
noncell_sef = scaler[14]
battery_sef = scaler[15];

In [5]:
input_amount = []
input_amount = vcat(up_cath, up_cell, cell_sef, up_noncell, noncell_sef, battery_sef) .* cell_demand
input_amount

MktV = price .* input_amount ;  # ton

In [6]:
input_amount

15-element Vector{Float64}:
  49670.291589732
  69512.470736544
  67696.846500888
  69382.78329114
 129687.44540400001
  72376.074329
   5239.896784
   1309.974196
  31111.887154999997
  60258.81301599999
 327493.549
   2258.5762
   1806.86096
 451715.24
 451715.24

### ECOLOGICAL

In [46]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [47]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)   # y[i,j,k] ship product k from i to j 
@variable(model, delta[1:ncty], Bin);

In [48]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end


# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end


# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end


# LCA: As=f
for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)


# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end

In [10]:
# 1. production emission (TES)
val_x = zeros(ncty, nproc)

for k in 1:nproc
    for i in 1:ncty
#         val_x[i,k] = regional_EF[i, k+1] - (price[k]/vGDP[i])*sink_c[i] - (price[k]/global_gdp)*global_sink
        val_x[i,k] = regional_EF[i, k+1] * (1 - sink_c[i]/emission_c[i] - es_ratio)
#         val_x[i,k] = regional_EF[i, k+1]
    end
end

proc_emi = sum(x .* val_x);

In [11]:
# 2. transpotration emission (TES, w/ supply)
trans_emi = 0

for i in 1:ncty
    for j in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        arc_emi = amount * distance[!, 2:end][i,j] * EF_trans
        arc_seq = arc_emi * (sink_c[j]/emission_c[j] + es_ratio)
        arc_net = arc_emi - arc_seq
        trans_emi += arc_net
#         trans_emi += arc_emi
    end
end

In [45]:
# slack = 1e-12
# proM = x * price
# proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 


# transD = Vector{AffExpr}(undef, ncty)
# for j in 1:ncty
#     Temi = 0
#     for i in 1:ncty
#         amount = sum(y[i,j,k] for k in 1:nproc)
#         Temi += amount * distance[!, 2:end][i,j] * EF_trans
#     end
#     transD[j] = Temi
# end

# # for i in 1:ncty
# #     @constraint(model, proM[i] <= vGDP[i])
# # end

# M = 1e15
# Allo_soc = proD ./ emission_c .* Dsoc
# # Allo_soc = proM ./ vGDP .* Dsoc
# SJ = proD + transD - Allo_soc;

# for i in 1:ncty
#     @constraint(model, SJ[i] >= (delta[i] - 1) * M)
#     @constraint(model, SJ[i] + slack <= delta[i] * M)
# end

In [13]:
@objective(model, Min, proc_emi+trans_emi);

In [14]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2265 rows, 13980 columns and 55800 nonzeros
Model fingerprint: 0xd1abdf61
Variable types: 13950 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+15]
  Objective range  [7e-03, 3e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-12, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2050 rows and 13173 columns
Presolve time: 0.05s
Presolved: 215 rows, 807 columns, 2175 nonzeros
Variable types: 804 continuous, 3 integer (3 binary)

Root relaxation: objective -8.194761e+06, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    -8194760.771 -8194760.8 

In [15]:
eco_opt_x = JuMP.value.(x)
eco_opt_y = JuMP.value.(y)
eco_opt_delta = JuMP.value.(delta)
eco_opt = JuMP.objective_value(model);

In [16]:
res_x = DataFrame(eco_opt_x, :auto)
rename!(res_x, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x, 1, :country => countries)

Row,country,Li,Co,Mn,Ni,cathode,graphite,PP,PE,Cu,Al,cell,PET,electronics,noncell,battery
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,6200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia,1770.29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brazil,2200.0,0.0,67696.8,67000.0,0.0,62376.1,5239.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Canada,500.0,3100.0,0.0,2382.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Chile,39000.0,0.0,0.0,0.0,0.0,0.0,0.0,1309.97,31111.9,0.0,0.0,2258.58,0.0,4.51715e5,0.0
6,China,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Colombia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Congo,0.0,66412.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [1]:
# CSV.write("/Users/bourg/Desktop/eco_val_1005.csv", res_x) 

### SOCIAL

In [17]:
model = Model(Gurobi.Optimizer);

Academic license - for non-commercial use only - expires 2023-11-27


In [18]:
#variables 
@variable(model, x[1:ncty, 1:nproc] >= 0)            # x[i,k] production amount of product k at location i
@variable(model, y[1:ncty, 1:ncty, 1:nproc] >= 0)   # y[i,j,k] ship product k from i to j 
@variable(model, delta[1:ncty], Bin);

In [19]:
# capacity constraint: x[i,k] <= capacity[i][k]
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, x[i,k] <= capacity[!, 2:end][i,k])
    end
end


# node output flow constraint
for k in 1:nproc
    for i in 1:ncty
        @constraint(model, sum(y[i,j,k] for j in 1:ncty) == x[i,k])
    end
end


# node output flow to market y[i,38, 15]
for i in 1:ncty
    for j in [e for e in 1:ncty if e != mkt_loc]
        @constraint(model, y[i,j,mkt_proc] == 0)
    end
end


# LCA: As=f
for k in 1:nproc-1
    @constraint(model, sum(x[i,k] for i in 1:ncty) == input_amount[k])
end

# final demand constraint
@constraint(model, sum(y[i,mkt_loc,mkt_proc] for i in 1:ncty) == cell_demand)


# cathode LCA constraints (index=5)
for k in cathode
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,5] * scaler[k])
    end
end


# cell LCA constraints (index=11)
for k in cell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,11] * scaler[k])
    end
end


# non cell LCA constraints (index=14)
for k in noncell
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,14] * scaler[k])
    end
end


# battery LCA constraints (index=15)
for k in battery
    for j in 1:ncty
        @constraint(model, sum(y[i,j,k] for i in 1:ncty) == x[j,15] * scaler[k])
    end
end

In [20]:
slack = 1e-12
# proM = x * price
proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1) 


transD = Vector{AffExpr}(undef, ncty)
for j in 1:ncty
    Temi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = Temi
end


# for i in 1:ncty
#     @constraint(model, proM[i] <= vGDP[i])
# end

M = 1e15
Allo_soc = proD ./ emission_c .* Dsoc
# Allo_soc = proM ./ vGDP .* Dsoc
SJ = proD + transD - Allo_soc;

for i in 1:ncty
    @constraint(model, SJ[i] >= (delta[i] - 1) * M)
    @constraint(model, SJ[i] + slack <= delta[i] * M)
end

In [21]:
@objective(model, Max, sum(delta));

In [22]:
JuMP.optimize!(model)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 2265 rows, 13980 columns and 55800 nonzeros
Model fingerprint: 0x72a00c41
Variable types: 13950 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [4e-03, 1e+15]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-12, 1e+15]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 2058 rows and 13268 columns
Presolve time: 0.05s
Presolved: 207 rows, 712 columns, 1984 nonzeros
Variable types: 709 continuous, 3 integer (3 binary)

Root relaxation: objective 3.000000e+01, 41 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0      30.0000000   30.00000  

In [23]:
soc_opt_x = JuMP.value.(x)
soc_opt_y = JuMP.value.(y)
soc_opt_delta = JuMP.value.(delta)
soc_opt = JuMP.objective_value(model);

In [24]:
res_x1 = DataFrame(soc_opt_x, :auto)
rename!(res_x1, ["x$i" => proc for (i, proc) in enumerate(process)])
insertcols!(res_x1, 1, :country => countries)

Row,country,Li,Co,Mn,Ni,cathode,graphite,PP,PE,Cu,Al,cell,PET,electronics,noncell,battery
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,Argentina,6200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Australia,19870.3,7000.0,0.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Brazil,2200.0,0.0,0.0,1382.78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.51715e5,0.0
4,Canada,500.0,3100.0,0.0,0.0,0.0,15000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Chile,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,China,19000.0,2200.0,0.0,0.0,1.29687e5,30126.1,0.0,0.0,0.0,0.0,3.27494e5,0.0,0.0,0.0,1.12929e5
7,Colombia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,Congo,0.0,40058.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Finland,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


---

In [25]:
x = soc_opt_x
y = soc_opt_y

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1);

In [26]:
# y[1,1,:]

In [27]:
# dis = ones(3,3) 
# dis[1,1] = 0
# dis[2,2] = 0
# dis[3,3] = 0;

# @variable(model, y2[1:3, 1:3, 1:2])

# transD = Vector{AffExpr}(undef, 3)
# for j in 1:3
#     Temi = 0
#     for i in 1:3
#         amount = sum(y2[i,j,k] for k in 1:2)
#         Temi += amount * dis[i,j] * 2
#     end
#     transD[j] = Temi
# end

# transD

In [28]:
transD = zeros(ncty)
for j in 1:ncty
    Temi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = Temi
end

In [29]:
transD

30-element Vector{Float64}:
      0.0
      0.0
   4065.076441123299
      0.0
      0.0
 460290.24235270347
      0.0
      0.0
      0.0
      0.0
 177080.52904828012
      0.0
      0.0
      ⋮
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
 298118.6840970912

In [34]:
Allo_soc = proD ./ emission_c .* Dsoc

30×1 Matrix{Float64}:
   1121.9571944883826
  28057.297615207633
   8224.407699072453
   2737.578987302439
      0.0
 144497.67657272262
      0.0
  25547.456053973416
      0.0
      0.0
  48000.106810120116
   4228.111832582232
  26895.271323787725
      ⋮
   2009.847413929412
      0.0
  85787.94577997028
    160.67544162467706
      0.0
      0.0
  45563.593165278595
      0.0
   7848.051087595
  18315.699603826903
      0.0
  86119.25750109706

In [36]:
SJ = proD + transD - Allo_soc

30×1 Matrix{Float64}:
  13210.141160911602
 339427.17283313477
  98208.29648582499
 143663.38424669756
      0.0
      2.8277143099873792e6
      0.0
 993362.7795489429
      0.0
      0.0
 896105.0297891888
  11043.575682117767
 422878.7575937923
      ⋮
   9533.195166070589
      0.0
 184352.81965482974
   1242.890186575323
      0.0
      0.0
 442930.6989432371
      0.0
  97556.430672805
 163724.48548298923
      0.0
      3.3001821760493964e6

In [38]:
res = 0
for i in SJ
    if i >= 0
        res += 1
    end
end
res

30

---

In [39]:
x = eco_opt_x
y = eco_opt_y

proD = (x .* Matrix(regional_EF[:,2:end])) * ones(nproc,1);

In [40]:
transD = zeros(ncty)
for j in 1:ncty
    Temi = 0
    for i in 1:ncty
        amount = sum(y[i,j,k] for k in 1:nproc)
        Temi += amount * distance[!, 2:end][i,j] * EF_trans
    end
    transD[j] = Temi
end

In [41]:
transD

30-element Vector{Float64}:
      0.0
      0.0
      0.0
      0.0
   1359.519955440268
      0.0
      0.0
      0.0
      0.0
 172298.2769216944
      0.0
      0.0
      0.0
      ⋮
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
 515860.48035834427

In [42]:
Allo_soc = proD ./ emission_c .* Dsoc

30×1 Matrix{Float64}:
   1121.957194488384
    323.86127472092306
  66934.81575366916
   2674.2293804581923
 114105.21709270366
      0.0
      0.0
  42354.82898826285
      0.0
  43667.438328610166
      0.0
      0.0
      0.0
      ⋮
 177645.89420219106
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
 106573.5811345179

In [43]:
SJ = proD + transD - Allo_soc

30×1 Matrix{Float64}:
  13210.141160911617
   3917.95811472147
 766190.0186271066
 140338.9070527408
 331217.69108638034
      0.0
      0.0
      1.6468845493740332e6
      0.0
 833800.957548343
      0.0
      0.0
      0.0
      ⋮
 842617.6873644468
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      0.0
      4.230948910219429e6

In [44]:
res = 0
for i in SJ
    if i >= 0
        res += 1
    end
end
res

30